In [1]:
import pandas as pd
cal = pd.read_csv('/data/linq/.qlib/qlib_data/cn_data/calendars/day.txt')
delta = 0
TODAY = str(cal.iloc[-1-delta,0])
YESTERDAY = str(cal.iloc[-2-delta,0])
# TODAY = str(cal.iloc[-2,0])
# YESTERDAY = str(cal.iloc[-3,0])

BT_START = '2024-10-15'

print(BT_START)
print(YESTERDAY)
print(TODAY)

2024-10-15
2025-02-20
2025-02-21


In [2]:
# BT_START = '2024-12-01'
# TODAY = '2024-12-09'

test_split = (TODAY, TODAY)
bt_split = (BT_START, YESTERDAY)
pos_txt = f'''\
date {TODAY}
cash 221866.00
SH605507 1000
SH603699 700
SH603398 1200
SH601963 2200
SH600104 900
SH600000 1900
SZ002991 300
SZ002938 500
SZ002851 300
SZ000877 3800
SZ000736 2400
'''

In [3]:
import qlib 
import pandas as pd
from pandas import Timestamp
import json
from qlib.constant import REG_CN
from qlib.utils import exists_qlib_data, init_instance_by_config
from qlib.workflow import R
from qlib.workflow.record_temp import SignalRecord, PortAnaRecord
from qlib.utils import flatten_dict
from qlib.data.filter import NameDFilter
from qlib.backtest.position import Position
from qlib.data import D

import copy
import pprint
import warnings
warnings.filterwarnings('ignore')

provider_uri = "/data/linq/.qlib/qlib_data/cn_data"  # target_dir
qlib.init(provider_uri=provider_uri, region=REG_CN)

from lilab.qlib.utils.tools import normalize_position_history, load_position_text, load_position_history, save_position_history, fill_price_position_history
from lilab.qlib.backtest.benchmark import BENCH_Step

/home/linq/anaconda3/envs/qlib39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2246156:MainThread](2025-02-22 05:50:13,520) INFO - qlib.Initialization - [config.py:416] - default_conf: client.


[2246156:MainThread](2025-02-22 05:50:13,524) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.


[2246156:MainThread](2025-02-22 05:50:13,525) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': PosixPath('/data/linq/.qlib/qlib_data/cn_data')}


In [4]:
import wk_db
import matplotlib
matplotlib.rcParams['font.family'] = ['Noto Sans CJK JP']

def add_info_df(df):
    stocks = df.index.values
    ilst = []
    for s in stocks:
        s = s[2:]+'.'+s[:2]
        df0 = wk_db.read_sql(f"SELECT sec_name, industry_name FROM a_share_market where windcode=='{s}' order by trade_dt;", db_loc='low_freq_db')
        df0 = pd.DataFrame(df0.iloc[-1,:]).T
        ilst.append(df0)
    df1 = pd.concat(ilst, axis=0)
    df1.index = df.index
    res = pd.concat([df1,df], axis=1)
    return res

Record position history

In [5]:
cur_pos = load_position_text(pos_txt)
norm_cur_pos = normalize_position_history(cur_pos)
norm_cur_pos = fill_price_position_history(norm_cur_pos)
try:
    pos_history = load_position_history()
except:
    pos_history = {}
pos_history.update(norm_cur_pos)
# print(pos_history)
save_position_history(pos_history)
dt, pos = next(iter(cur_pos.items()))
account = pos.position
dt, norm_pos = next(iter(norm_cur_pos.items()))
norm_account = pos.position
hold_lst = pos.get_stock_list()
print(dt.date())
print(account)
print(norm_pos)
print(hold_lst)

2025-02-21
{'SH605507': {'amount': 1000}, 'SH603699': {'amount': 700}, 'SH603398': {'amount': 1200}, 'SH601963': {'amount': 2200}, 'SH600104': {'amount': 900}, 'SH600000': {'amount': 1900}, 'SZ002991': {'amount': 300}, 'SZ002938': {'amount': 500}, 'SZ002851': {'amount': 300}, 'SZ000877': {'amount': 3800}, 'SZ000736': {'amount': 2400}, 'cash': 221866.0}
{'_settle_type': 'None', 'position': {'SH603699': {'amount': 13134, 'price': 1.4331074953079224}, 'SZ002938': {'amount': 10398, 'price': 2.054215431213379}, 'SZ000877': {'amount': 35198, 'price': 0.5819060206413269}, 'SH601963': {'amount': 27938, 'price': 0.6882189512252808}, 'SZ000736': {'amount': 13036, 'price': 1.1985085010528564}, 'SH600000': {'amount': 9651, 'price': 2.0257458686828613}, 'SH605507': {'amount': 43713, 'price': 0.45043498277664185}, 'SH600104': {'amount': 10195, 'price': 1.5270837545394897}, 'SZ002851': {'amount': 2252, 'price': 8.352743148803711}, 'SZ002991': {'amount': 14955, 'price': 1.540541172027588}, 'SH603398':

In [6]:

pos_history = load_position_history()
hold_day = {}
_norm_pos = norm_pos.__dict__['position']
print(_norm_pos)
dt_lst = sorted(list(pos_history.keys()))[::-1]
for k, v in _norm_pos.items():
    if not isinstance(v, dict):
        continue
    cnt = 0
    for dt in dt_lst:
        if str(dt.date())==TODAY:
            # print(dt.date(), TODAY)
            continue
        tmp_pos = pos_history[dt]
        if k not in tmp_pos.get_stock_list():
            break
        cnt += 1
    # print(k, cnt)
    hold_day[k] = cnt
print(hold_day)

{'SH603699': {'amount': 13134, 'price': 1.4331074953079224}, 'SZ002938': {'amount': 10398, 'price': 2.054215431213379}, 'SZ000877': {'amount': 35198, 'price': 0.5819060206413269}, 'SH601963': {'amount': 27938, 'price': 0.6882189512252808}, 'SZ000736': {'amount': 13036, 'price': 1.1985085010528564}, 'SH600000': {'amount': 9651, 'price': 2.0257458686828613}, 'SH605507': {'amount': 43713, 'price': 0.45043498277664185}, 'SH600104': {'amount': 10195, 'price': 1.5270837545394897}, 'SZ002851': {'amount': 2252, 'price': 8.352743148803711}, 'SZ002991': {'amount': 14955, 'price': 1.540541172027588}, 'SH603398': {'amount': 7699, 'price': 2.3549671173095703}, 'cash': 221866.0, 'now_account_value': 432170.33118397}
{'SH603699': 6, 'SZ002938': 6, 'SZ000877': 10, 'SH601963': 6, 'SZ000736': 2, 'SH600000': 6, 'SH605507': 8, 'SH600104': 2, 'SZ002851': 5, 'SZ002991': 7, 'SH603398': 3}


In [7]:
def simp_eval(res_df, k, t=None, day=None):
    # print(res_df)
    if day:
        date = str(cal.iloc[-day-1,0])
        res_df = res_df[res_df.index.get_level_values('datetime') >= date]
    # Step 1: Group by multiple indices
    grouped = res_df.groupby(['datetime'])

    # Step 2: Sort by 'value' within each group in descending order
    sorted_df = grouped.apply(lambda x: x.sort_values(by='score', ascending=False))
    sorted_df['win'] = sorted_df['label'] * sorted_df['score'] > 0
    # Step 3: Select the top 5 rows for each group
    # top_k_per_group = sorted_df.groupby(['datetime']).head(k)
    top_k_per_group = sorted_df.groupby(['datetime']).apply(lambda x: x.nlargest(k, 'score'))#.reset_index(level=1, drop=1)
    if t:
        # top_k_per_group = top_k_per_group.tail(t)
        top_k_per_group = top_k_per_group.groupby(['datetime']).apply(lambda x: x.nsmallest(t, 'score'))
    # print(top_k_per_group)
    top_k_ic = top_k_per_group['label'].corr(top_k_per_group['score'])
    s, n = top_k_per_group['win'].sum(), len(top_k_per_group)
    # print(s, n)
    top_k_win = 1.0 * top_k_per_group['win'].sum() / len(top_k_per_group)
    return top_k_ic, top_k_win 

def run_model(model, path='/home/linq/finance/qniverse/log'):
    res_df = pd.read_csv(f'{path}/{model}_backtest_return_{TODAY}.csv', index_col=['datetime','instrument'])

    def eval_day(day=None):
        top5ic, top5win = simp_eval(res_df, 5, day=day)
        top10ic, top10win = simp_eval(res_df, 10, 5, day=day)
        # top15ic, top15win = simp_eval(res_df, 15, 5, day=day)
        top20ic, top20win = simp_eval(res_df, 20, 10, day=day)
        # top25ic, top25win = simp_eval(res_df, 25, 5, day=day)
        top30ic, top30win = simp_eval(res_df, 30, 10, day=day)
        top40ic, top40win = simp_eval(res_df, 40, 10, day=day)
        top50ic, top50win = simp_eval(res_df, 50, 10, day=day)
        top60ic, top60win = simp_eval(res_df, 60, 10, day=day)
        top70ic, top70win = simp_eval(res_df, 70, 10, day=day)
        top80ic, top80win = simp_eval(res_df, 80, 10, day=day)        
        top90ic, top90win = simp_eval(res_df, 90, 10, day=day)
        top100ic, top100win = simp_eval(res_df, 100, 10, day=day)
        top110ic, top110win = simp_eval(res_df, 110, 10, day=day)
        top120ic, top120win = simp_eval(res_df, 120, 10, day=day)
        top130ic, top130win = simp_eval(res_df, 130, 10, day=day)
        top140ic, top140win = simp_eval(res_df, 140, 10, day=day)
        top150ic, top150win = simp_eval(res_df, 150, 10, day=day)
        topic, topwin = simp_eval(res_df, 50, day=day)
        # return [topic, topwin, top5ic, top5win, top10ic, top10win, top15ic, top15win, top20ic, top20win, top25ic, top25win, top30ic, top30win]
        return [topwin, top5win, top10win, top20win, top30win, top40win, top50win, top60win, top70win, top80win, top90win, top100win, top110win, top120win, top130win, top140win, top150win], [topic, top5ic, top10ic, top20ic, top30ic, top40ic, top50ic, top60ic, top70ic, top80ic, top90ic, top100ic, top110ic, top120ic, top130ic, top140ic, top150ic]
    day1metrics = eval_day(2)
    day5metrics = eval_day(6)
    day10metrics = eval_day(11)
    day20metrics = eval_day(21)
    
    pred_df = pd.read_csv(f'{path}/{model}_tomorrow_return_{TODAY}.csv', index_col=['datetime','instrument'])

    return day1metrics, day5metrics, day10metrics, day20metrics, pred_df, res_df

# Run

In [8]:
info_win_dict1, info_ic_dict1 = {}, {}
info_win_dict5, info_ic_dict5 = {}, {}
info_win_dict10, info_ic_dict10 = {}, {}
info_win_dict20, info_ic_dict20 = {}, {}
pred_lst = []

name_lst =  ['MENSEMBLE0', 'MENSEMBLE']

for model in name_lst:
    NAME = model
    print(model)
    day1metrics, day5metrics, day10metrics, day20metrics, pred_df, res_df = run_model(model)
    info_win_dict1[NAME], info_ic_dict1[NAME] = day1metrics
    info_win_dict5[NAME], info_ic_dict5[NAME] = day5metrics
    info_win_dict10[NAME], info_ic_dict10[NAME] = day10metrics
    info_win_dict20[NAME], info_ic_dict20[NAME] = day20metrics
    # for k in [1, 5, 10, 20]:
    #     eval(f'    info_win_dict{k}[NAME], info_ic_dict{k}[NAME] = day{k}metrics')
    _df = pred_df.loc[:,'score']
    _df.name = model
    # print(_df)
    pred_lst.append(_df)

MENSEMBLE0


MENSEMBLE


In [9]:
from matplotlib.colors import LinearSegmentedColormap

colors = ['green', 'white', 'red']
n_neg = 256 // 2
n_pos = 256 - n_neg
cmap = LinearSegmentedColormap.from_list('custom_red_green', colors, N=256)

In [10]:
pred_df = pd.concat(pred_lst, axis=1)
# pred_df
# # pred_df = pred_df.loc[:, ~pred_df.columns.duplicated()]
pred_df.columns = ['return 0', 'return 1']
_pred_df = pred_df 

pred_df['sum'] = _pred_df['return 0'] + _pred_df['return 1']
# pred_df['std'] = _pred_df.std(axis=1)

rank_df = pred_df.droplevel(0).sort_values(by='return 1', ascending=False)
fields = ['$close/$factor']
_df = D.features(rank_df.index.values, fields, start_time=TODAY, end_time=TODAY, freq='day')
# print(_df)
_df.columns = ['close']
_df = _df.droplevel(1)
rank_df = pd.concat([_df, rank_df],axis=1,join='inner')
report_top = rank_df
report_top = report_top.sort_values(by='return 1', ascending=False)


# hold_lst
pos_rank = rank_df.sort_values(by='return 1', ascending=True)
pos_rank = pos_rank[pos_rank.index.isin(hold_lst)]

hd = [hold_day[s] for s in pos_rank.index.values]
pos_rank['HD'] = hd
report_pos = pos_rank
report_pos = add_info_df(report_pos)
color_lst = ['return 0', 'return 1', 'sum']
# color_lst
raw_sty_report_pos = report_pos.style.background_gradient(cmap=cmap, vmin=-0.05, vmax=0.05, subset=color_lst, axis=None)
raw_sty_report_pos

,sec_name,industry_name,close,return 0,return 1,sum,HD
instrument,,,,,,,
SZ002851,麦格米特,电力设备及新能源(中信),62.709999,-0.019466,-0.119873,-0.139339,5
SH601963,重庆银行,银行(中信),8.740000,0.005823,-0.014519,-0.008696,6
SH600000,浦发银行,银行(中信),10.290000,0.007923,-0.007070,0.000853,6
SZ002991,甘源食品,食品饮料(中信),76.800003,0.018491,0.009463,0.027954,7
SZ002938,鹏鼎控股,电子(中信),42.720001,-0.000043,0.010107,0.010063,6
SH605507,国邦医药,医药(中信),19.689999,0.007560,0.028670,0.036230,8
SH600104,上汽集团,汽车(中信),17.299999,0.017471,0.038812,0.056284,2
SZ000877,天山股份,建材(中信),5.390000,0.014953,0.043826,0.058779,10
SH603699,纽威股份,机械(中信),26.889999,0.013371,0.055800,0.069171,6


In [11]:
df0 = report_top.head(50)
df0 = add_info_df(df0)
sty_report_top0 = df0.style.background_gradient(cmap=cmap, axis=None, vmin=-0.05, vmax=0.05, subset=color_lst)
# sty_report_top1 = report_top.head(20).tail(10).style.background_gradient(cmap=cmap, axis=0, subset=color_lst)
# sty_report_top2 = report_top.head(30).tail(10).style.background_gradient(cmap=cmap, axis=0, subset=color_lst)
# sty_report_top3 = report_top.head(40).tail(10).style.background_gradient(cmap=cmap, axis=0, subset=color_lst)
# sty_report_top4 = report_top.head(50).tail(10).style.background_gradient(cmap=cmap, axis=0, subset=color_lst)
# sty_report_top5 = report_top.head(60).tail(10).style.background_gradient(cmap=cmap, axis=0, subset=color_lst)
# sty_report_top6 = report_top.head(70).tail(10).style.background_gradient(cmap=cmap, axis=0, subset=color_lst)
# sty_report_top7 = report_top.head(80).tail(10).style.background_gradient(axis=0, subset=color_lst)
# sty_report_top8 = report_top.head(90).tail(10).style.background_gradient(axis=0, subset=color_lst)
# sty_report_top9 = report_top.head(100).tail(10).style.background_gradient(axis=0, subset=color_lst)
# sty_report_top10 = report_top.head(110).tail(10).style.background_gradient(axis=0, subset=color_lst)
# sty_report_top11 = report_top.head(120).tail(10).style.background_gradient(axis=0, subset=color_lst)
# sty_report_top12 = report_top.head(130).tail(10).style.background_gradient(axis=0, subset=color_lst)
# sty_report_top13 = report_top.head(140).tail(10).style.background_gradient(axis=0, subset=color_lst)
# sty_report_top14 = report_top.head(150).tail(10).style.background_gradient(axis=0, subset=color_lst)

In [12]:
sty_report_top0

,sec_name,industry_name,close,return 0,return 1,sum
instrument,,,,,,
SZ002929,润建股份,通信(中信),68.460007,0.632150,0.194520,0.826670
SH603398,沐邦高科,电力设备及新能源(中信),15.110001,0.068389,0.180965,0.249354
SZ000736,中交地产,房地产(中信),6.510000,0.058743,0.159244,0.217987
SZ002126,银轮股份,汽车(中信),30.249998,0.585971,0.145477,0.731449
SZ002127,南极电商,商贸零售(中信),4.340000,0.034306,0.115799,0.150105
SH601069,西部黄金,有色金属(中信),12.900000,0.009455,0.109730,0.119185
SH603881,数据港,计算机(中信),40.369999,0.322472,0.094846,0.417318
SH600595,中孚实业,有色金属(中信),3.520000,0.010134,0.088047,0.098181
SH600489,中金黄金,有色金属(中信),12.630000,0.001871,0.085092,0.086963


In [13]:
import dataframe_image as dfi

infodf = pd.DataFrame({'label': ['Model update date', 'Prediction generation date'],
                       'date': ['2024-12-11', TODAY]})

dfi.export(infodf, './tmp/ensemble_private_all_info.png',table_conversion='matplotlib')

# dfi.export(sty_report_info5, './tmp/ensemble_private_all_performance5.png',table_conversion='matplotlib')
# dfi.export(sty_report_info10, './tmp/ensemble_private_all_performance10.png',table_conversion='matplotlib')
# dfi.export(sty_report_info20, './tmp/ensemble_private_all_performance20.png',table_conversion='matplotlib')
dfi.export(sty_report_top0, './tmp/ensemble_private_all_top0.png',table_conversion='matplotlib')
# dfi.export(sty_report_top1, './tmp/ensemble_private_all_top1.png',table_conversion='matplotlib')
# dfi.export(sty_report_top2, './tmp/ensemble_private_all_top2.png',table_conversion='matplotlib')
# dfi.export(sty_report_top3, './tmp/ensemble_private_all_top3.png',table_conversion='matplotlib')
# dfi.export(sty_report_top4, './tmp/ensemble_private_all_top4.png',table_conversion='matplotlib')
# dfi.export(sty_report_top5, './tmp/ensemble_private_all_top5.png',table_conversion='matplotlib')
# dfi.export(sty_report_top6, './tmp/ensemble_private_all_top6.png',table_conversion='matplotlib')
# dfi.export(sty_report_top7, './tmp/ensemble_private_all_top7.png',table_conversion='matplotlib')
# dfi.export(sty_report_top8, './tmp/ensemble_private_all_top8.png',table_conversion='matplotlib')
# dfi.export(sty_report_top9, './tmp/ensemble_private_all_top9.png',table_conversion='matplotlib')
# dfi.export(sty_report_top10, './tmp/ensemble_private_all_top10.png',table_conversion='matplotlib')
# dfi.export(sty_report_top11, './tmp/ensemble_private_all_top11.png',table_conversion='matplotlib')
# dfi.export(sty_report_top12, './tmp/ensemble_private_all_top12.png',table_conversion='matplotlib')
# dfi.export(sty_report_top13, './tmp/ensemble_private_all_top13.png',table_conversion='matplotlib')
# dfi.export(sty_report_top14, './tmp/ensemble_private_all_top14.png',table_conversion='matplotlib')
# dfi.export(sty_report_pos, './tmp/ensemble_private_all_watch.png',table_conversion='matplotlib')
dfi.export(raw_sty_report_pos, './tmp/ensemble_private_raw_all_watch.png',table_conversion='matplotlib')

In [14]:

for k in [1,5,10,20]:
    sty_report_win_info = eval(f"pd.DataFrame(info_win_dict{k}, index=['TOP50WIN', '0-5', '5-10', '10-20', '20-30', '30-40', '40-50', '50-60', '60-70', '70-80', '80-90', '90-100', '100-110', '110-120', '120-130', '130-140', '140-150'])").style.background_gradient(axis=None)
    sty_report_ic_info = eval(f"pd.DataFrame(info_ic_dict{k}, index=['TOP50IC', '0-5', '5-10', '10-20', '20-30', '30-40', '40-50', '50-60', '60-70', '70-80', '80-90', '90-100', '100-110', '110-120', '120-130', '130-140', '140-150'])").style.background_gradient(axis=None)
    dfi.export(sty_report_win_info, f'./tmp/ensemble_private_all_win{k}.png',table_conversion='matplotlib')
    dfi.export(sty_report_ic_info, f'./tmp/ensemble_private_all_ic{k}.png',table_conversion='matplotlib')

In [15]:
tot_invest = 231890

dt_lst = sorted(list(pos_history.keys()))
cash, sv, tot = [], [], []
for dt in dt_lst:
    pos: Position = pos_history[dt]
    cash.append(pos.get_cash())
    sv.append(pos.calculate_stock_value())
    tot.append(pos.calculate_value())
res = pd.DataFrame({'Total Value':cash}, index=[d.date() for d in dt_lst])
res['Balance'] = res['Total Value'] - res['Total Value'].shift(1) 
res['Return'] = res['Balance'] / res['Total Value'].shift(1) 
res['Tot Balance'] = res['Total Value'] - tot_invest
res['Tot Return'] = res['Tot Balance'] / tot_invest
res

,Total Value,Balance,Return,Tot Balance,Tot Return
2025-01-27,231890.0,NaN,NaN,0.0,0.000000
2025-02-05,208683.0,-23207.0,-0.100078,-23207.0,-0.100078
2025-02-06,209094.0,411.0,0.001969,-22796.0,-0.098305
2025-02-07,213032.0,3938.0,0.018834,-18858.0,-0.081323
2025-02-10,217626.0,4594.0,0.021565,-14264.0,-0.061512
2025-02-11,216700.0,-926.0,-0.004255,-15190.0,-0.065505
2025-02-12,220518.0,3818.0,0.017619,-11372.0,-0.049040
2025-02-13,222893.0,2375.0,0.010770,-8997.0,-0.038799
2025-02-14,222809.0,-84.0,-0.000377,-9081.0,-0.039161
2025-02-17,222809.0,0.0,0.000000,-9081.0,-0.039161


In [16]:
dfi.export(res, './tmp/private_account.png',table_conversion='matplotlib')

In [17]:
infodf = pd.DataFrame({'label': ['Update date', 'Backtest start date'],
                    'value': [TODAY, BT_START]})

dfi.export(infodf, f'./tmp/summary_info.png',table_conversion='matplotlib')